<a href="https://colab.research.google.com/github/muhammadnouman911/Face-Detection-Classification-Regression/blob/main/face_recognition_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface scikit-learn opencv-python

In [ ]:
import os
from deepface import DeepFace
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle


25-08-16 06:38:18 - Directory /root/.deepface has been created
25-08-16 06:38:18 - Directory /root/.deepface/weights has been created


In [ ]:
# Step 2: Set Dataset Path
dataset_path = "/content/drive/MyDrive/dataset/faces"  # Change this to your dataset folder path


In [ ]:
# Step 3: Load Images & Extract Embeddings (No detection, for cropped faces)
def extract_embeddings(dataset_path):
    X = []  # embeddings
    y = []  # labels

    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_folder):
            continue

        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            try:
                embedding = DeepFace.represent(
                    img_path=img_path,
                    model_name='Facenet512',
                    detector_backend='skip',      # Skip detection
                    enforce_detection=False       # Allow all images
                )
                X.append(embedding[0]['embedding'])
                y.append(person_name)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
    return np.array(X), np.array(y)

print("Extracting embeddings...")
X, y = extract_embeddings(dataset_path)
print(f"Extracted {len(X)} embeddings.")

Extracting embeddings...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5


25-08-16 06:38:41 - 🔗 facenet512_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5 to /root/.deepface/weights/facenet512_weights.h5...


100%|██████████| 95.0M/95.0M [00:00<00:00, 243MB/s]


Extracted 6169 embeddings.


In [ ]:
# Step 4: Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [ ]:
# Step 5: Train Classifier
clf = SVC(kernel='linear', probability=True)
clf.fit(X, y_encoded)


SVC(kernel='linear', probability=True)

In [ ]:
# Step 6: Save Model & Encoder
with open("face_classifier.pkl", "wb") as f:
    pickle.dump(clf, f)

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("Model training complete and saved.")


Model training complete and saved.


In [ ]:
# Step 7: Prediction Function
def predict_face(img_path):
    embedding = DeepFace.represent(
        img_path=img_path,
        model_name='Facenet512',
        detector_backend='skip',
        enforce_detection=False
    )[0]['embedding']

    with open("face_classifier.pkl", "rb") as f:
        clf = pickle.load(f)
    with open("label_encoder.pkl", "rb") as f:
        label_encoder = pickle.load(f)

    probs = clf.predict_proba([embedding])[0]
    class_index = np.argmax(probs)
    class_label = label_encoder.inverse_transform([class_index])[0]
    confidence = probs[class_index] * 100

    return class_label, confidence


In [ ]:
# Step 8: Example Prediction
test_img = "/content/drive/MyDrive/dataset/faces/AvWWVOgaMlk_90.0_93.566667/frame_0001.jpg"  # Change to your test image
label, conf = predict_face(test_img)
print(f"Predicted: {label} ({conf:.2f}% confidence)")


Predicted: AvWWVOgaMlk_90.0_93.566667 (93.26% confidence)
